In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer
from wordvecs import WordVectors, EmbeddingLayerLayer
import json

theano.config.floatX = 'float32'
theano.config.linker = 'c'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
with open('/data/matthew/external-wiki1.json') as f:
    queries = json.load(f)['queries']

In [5]:
len(queries)

9915

In [6]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [7]:
8917/9915.

0.8993444276348966

In [8]:

wordvectors = WordVectors(
    fname="/data/matthew/GoogleNews-vectors-negative300.bin",
    negvectors=False
)

In [9]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
    page_redirects = json.load(f)

In [10]:
from wikireader import WikiRegexes, WikipediaReader

In [11]:
def PreProcessedQueries(wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wordvec.tokenize(sur)
            for link in v['vals'].keys():
                wordvec.tokenize(link)
                tt = WikiRegexes.convertToTitle(link)
                #self.wordvecs.tokenize(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

    class GetWikipediaWords(WikipediaReader, WikiRegexes):

        def readPage(ss, title, content):
            tt = ss.convertToTitle(title)
            if tt in queried_pages:
                cnt = ss._wikiToText(content)
                page_content[tt] = wordvec.tokenize(cnt)

    GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        
        
    return PreProcessedQueriesCls

In [12]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-test.xml')

In [13]:
class EinsumOpt(theano.gof.Op):

    __props__ = ("operation", )

    def __init__(self, operation, shape):
        super(EinsumOpt, self).__init__()
        self.operation = operation
        self.shape = shape
        # TODO: parse the operation string to determine the size of the output

    def __str__(self):
        return "Einsum{%s}" % self.operation

    def make_node(self, *args):
        out = theano.tensor.tensor(args[0].dtype, self.shape)
        return theano.gof.Apply(self, list(args), [out])

    def perform(self, node, inp, out):
        z, = out
        z[0] = np.einsum(self.operation, *inp)

    def c_code(self, node, name, inp, out, sub):
        operation = self.operation
        num_inputs = len(inp)

        input_str = "{" + ', '.join(['(PyArrayObject*){}'.format(i) for i in  inp]) + ', NULL }'
        first_inp = inp[0]
        z, = out
        fail = sub['fail']

        c_code = """
        // TODO: check the inputs I guess and stuff
        {
          PyObject* ret;
          PyArrayObject* input_args[] = %(input_str)s ;
          ret = (PyObject*) PyArray_EinsteinSum("%(operation)s", %(num_inputs)s, input_args, PyArray_DescrFromType(PyArray_TYPE(%(first_inp)s)), NPY_CORDER, NPY_SAFE_CASTING, %(z)s);
          if(ret != NULL && %(z)s == NULL) {
            if(!PyArray_Check(ret)) {
              %(fail)s ;
            } else {
              %(z)s = (PyArrayObject*) ret;
            }
          }
        }
        """

        return c_code % locals()


In [14]:
# class OptedBatchedDot(theano.gof.Op):
    
#     def __inti__(self):
#         pass
    
#     def __str__(self):
#         return "OptedBatchedDot"
    
#     def make_node(self, x, y):
#         return theano.gof.Apply(self, [x,y], [theano.tensor.vector()])
    
# #     def perform(self, node, inp, out):
# #         x,y = inp
# #         z, = out
# #         result, updates = theano.scan(
# #             fn=lambda x_mat, y_mat:
# #             theano.tensor.dot(x_mat, y_mat),
# #             outputs_info=None,
# #             sequences=[x, y],
# #             non_sequences=None
# #         )
        
# #         return result

#     def c_code_cache_version(self):
#         return (1,0)
    
#     def c_code(self, node, name, inp, out, sub):
#         x, y = inp
#         z, = out
                         
#         dtype_x = node.inputs[0].dtype
        
#         fail = sub['fail']
                         
#         c_code = """
#         // ensure that the output size is correctly allocated
#         if(NULL = %(z)s || PyArray_DIMS(%(x)s)[0] != PyArray_DIMS(%(z)s)[0]) {
#           Py_XDECREF(%(z)s);
#           %(z)s = (PyArrayObject*)PyArray_EMPTY(1, 
#             PyArray_DIMS(%(x)s),
#             PyArray_TYPE(%(x)s),
#             0);
          
#           if(!%(z)s) {
#             %(fail)s;
#           }
#         }
        
#         {
#           npy_%(dtype_x)s x_data = (npy_%(dtype_x))) PyArray_DATA()
        
#         }
#         """
                         
                         
#         return c_code % locals()
        
        
# pBatched_dot = OptedBatchedDot()

In [16]:
class EntityVectorLinkExp(basePreProcessedQueries):
    
    batch_size = 20000
    
    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.num_words_to_use_conv = 3
        
        self._setup()       
        
    def _setup(self):
        self.x_document_input = T.imatrix('x_sent')
        self.x_surface_text_input = T.imatrix('x_surface')
        self.x_target_input = T.imatrix('x_target')
        self.x_document_id = T.ivector('x_sent_id')
        self.x_link_id = T.ivector('x_link_id')
        self.y_score = T.vector('y')
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        
        self.document_l = lasagne.layers.InputLayer(
            (None,self.sentence_length), 
            input_var=self.x_document_input
        )
    
        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.document_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=350,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='document_conv1',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.document_max_l = lasagne.layers.Pool2DLayer(
            self.document_conv1_l,
            name='document_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='max',
        )

        self.document_dens1 = lasagne.layers.DenseLayer(
            self.document_max_l,
            num_units=300,
            name='doucment_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_drop1 = lasagne.layers.DropoutLayer(
            self.document_dens1,
            p=.25,
        )
        
        document_output_length = 250
        
        self.document_dens2 = lasagne.layers.DenseLayer(
            self.document_drop1,
            num_units=document_output_length,
            name='document_dens2',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_output = lasagne.layers.get_output(self.document_dens2)
                
        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length), 
            input_var=self.x_surface_text_input
        )
        
        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=350,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_conv1_l,
            name='surface_dens1',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.surface_drop1 = lasagne.layers.DropoutLayer(
            self.surface_dens1,
            p=.25,
        )
        
        self.surface_dens2 = lasagne.layers.DenseLayer(
            self.surface_drop1,
            name='surface_dens2',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_aligned_l = InputLayer(
            (None, document_output_length),
            input_var=self.document_output[self.x_document_id,:]
        )
        
        self.source_l = lasagne.layers.ConcatLayer(
            [self.document_aligned_l, self.surface_dens2]
        )
        
        self.source_dens1 = lasagne.layers.DenseLayer(
            self.source_l,
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )
        
        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=300,
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_out = lasagne.layers.get_output(self.source_dens2)
        
        self.target_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length), 
            input_var=self.x_target_input
        )
        
        self.target_embedding_l = EmbeddingLayer(
            self.target_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_embedding_l,
            name='target_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_conv1_l,
            name='target_dens1',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )
        
        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_dens2',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_out = lasagne.layers.get_output(self.target_dens2)
        
        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]
               
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        #self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()
        self.dotted_vectors = EinsumOpt('ij,ji->i', (False, ))(self.target_out, self.source_aligned_l)
        
        self.res_l = self.dotted_vectors / (self.target_out.norm(2, axis=1) * self.source_aligned_l.norm(2, axis=1))
        
        self.gg_func = theano.function(
            [self.x_document_input,
             self.x_surface_text_input, self.x_document_id,
             self.x_target_input, self.x_link_id, self.y_score],
            [self.target_out, self.dotted_vectors, self.res_l],
            on_unused_input='ignore',
        )
                
        self.all_params = (
            lasagne.layers.get_all_params(self.target_dens2) + 
            lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.document_dens2)
        )
        
        self.loss_vec = T.nnet.binary_crossentropy(T.clip(self.res_l, .001, .999), self.y_score)
        
        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)
        
        self.train_func = theano.function(
            [self.x_document_input,
             self.x_surface_text_input, self.x_document_id,
             self.x_target_input, self.x_link_id, self.y_score],
            [self.res_l, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates
        )
        
        self.test_func = theano.function(
            [self.x_document_input,
             self.x_surface_text_input, self.x_document_id,
             self.x_target_input, self.x_link_id, self.y_score],
            [self.res_l, self.loss_vec.sum(), self.loss_vec],
        )
        
    def reset_accums(self):
        self.current_documents = []
        self.current_surface_text = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_id = []
        self.current_target_goal = []
        self.learning_targets = []
        
    def compute_batch(self, isTraining=True):
        if isTraining:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        
        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] is not isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_text)
                self.current_surface_text.append(self.wordvecs.tokenize(surtxt))
                for target in targets['vals'].keys():
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    cnt = self.page_content.get(WikiRegexes.convertToTitle(target), [0]*self.sentence_length)
                    self.current_target_input.append(cnt)  # page_content already tokenized
                    self.current_target_id.append(surid)
                    self.current_target_goal.append(isGold)
                    self.learning_targets.append((targets, target))
            if len(self.current_target_id) > self.batch_size:
                return
                self.run_batch(func)
        
        if len(self.current_target_id) > 0:
            return
            self.run_batch(func)
            
        return self.total_loss / self.total_links
        
    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            self.current_documents,
            self.current_surface_text, self.current_link_id,
            self.current_target_input, self.current_target_id, self.current_target_goal
        )
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            l[0]['vals'][ l[1] ] = res_vec[i]
        self.reset_accums()
        

        
queries_exp = EntityVectorLinkExp() 

AttributeError: 'EinsumOpt' object has no attribute 'grad'

TypeError: function_dump() takes at least 2 arguments (1 given)

In [ ]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_score],
            [queries_exp.target_out, queries_exp.source_aligned_l, 
             T.dot(queries_exp.target_out, queries_exp.source_aligned_l.T).diagonal(),
             queries_exp.target_out.norm(2, axis=1) * queries_exp.source_aligned_l.norm(2, axis=1),
             T.batched_dot(queries_exp.target_out, queries_exp.source_aligned_l)],
        #[queries_exp.res_l, queries_exp.loss_vec.sum(), queries_exp.loss_vec],
    on_unused_input='ignore'
        )

In [ ]:
gg_res = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_text, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [ ]:
gg_res[0].shape, gg_res[1].shape, gg_res[2].shape, gg_res[3].shape, gg_res[4].shape

In [ ]:
np.inner(gg_res[0], gg_res[1]).shape

In [ ]:
gg_res[0] * gg_res[1]

In [ ]:
aa = np.dot(gg_res[0], gg_res[1].T)

In [ ]:
aa.diagonal().shape

In [ ]:
gg_res[2]

In [ ]:
len(queries_exp.queried_pages)

In [ ]:
len(queries_exp.page_content)

In [ ]:
len(queries_exp.current_surface_text)

In [ ]:
len(queries_exp.current_link_id)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
exp_results = []

for i in xrange(15):
    exp_results.append((i, queries_exp.compute_batch()))

In [ ]:
exp_results

In [17]:
theano.config.openmp

False